In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pairprog.objectstore import * 
from pairprog.assistant import * 
from pairprog.assistant  import logger as as_logger

In [3]:
import logging
logging.basicConfig()
as_logger.setLevel(logging.DEBUG)

In [4]:
rc = ObjectStore.new(name='barker_minio', bucket='agent')

In [29]:
import typesense
ts = typesense.Client(
            {
                "api_key": "xyz",
                "nodes": [{"host": "barker", "port": "8108", "protocol": "http"}],
                "connection_timeout_seconds": 1,
            }
        )

tool = PPTools(ts, rc.sub('pptools'), Path('/Volumes/Cache/scratch'))

assis = Assistant(tool, cache=rc)

messages = [
    {"role": "user", "content": 'tell me a joke'}
]

def dump(o):
    import json 
    print( json.dumps(o.model_dump(), indent=2))
    
len(assis.tokenizer.encode(json.dumps(tool.specification())))

1045

In [6]:
for t in tool.specification():
    print(t['function']['name'])

execute_code
faster_and_cheaper
get_document
read
search_documents
shell
smarter
store_document
web_search
write


In [7]:
def get_sessions():
    
    sessions = set(e.split('/')[0] for e in list(rc.sub('session').list()))
    return list(reversed(sorted(sessions)))

def get_last_src():

    last_session  = get_sessions()[0]
    src = rc.sub('session/'+last_session)
    return src

def dump_sessions():
    last_session  = get_sessions()[-3]
    src = rc.sub('session/'+last_session)
    for e in src['responses']:
        print(e)
        print('-----')

src = get_last_src()

for e in src['messages']:
    print(e)
    print('-----')


{'role': 'user', 'content': 'get smarter, then answer this question: Which program beat GPT-3.5 in the Turing Test'}
-----
{'role': 'assistant', 'content': ''}
-----
{'content': '', 'function_call': None, 'role': 'assistant', 'tool_calls': [{'index': 0, 'id': 'call_3BpUK9gBb3WIhFfPRoalK45b', 'function': {'arguments': '{"text":"Which program beat GPT-3.5 in the Turing Test?"}', 'name': 'smarter'}, 'type': 'function'}]}
-----
{'tool_call_id': 'call_3BpUK9gBb3WIhFfPRoalK45b', 'role': 'tool', 'name': 'smarter', 'content': 'I am now smarter'}
-----
{'role': 'assistant', 'content': 'There isn\'t a clear answer to which specific program "beat" GPT-3.5 in the Turing Test—partly because the Turing Test is not a standardized benchmark and partly because there are many interpretations of what "beating" the test means. It\'s pivotal to understand that the Turing Test, proposed by Alan Turing in his 1950 paper "Computing Machinery and Intelligence," is a qualitative measure of a machine\'s ability 

In [12]:
assis.token_limit

12289

In [64]:
tool.wikipedia_search('Thomas Edison')

[{'page_name': 'Thomas Edison',
  'pageid': '29778',
  'store_document_uri': 'wiki:29778',
  'url': 'https://en.wikipedia.org/wiki/Thomas_Edison',
  'summary': 'Thomas Alva Edison (February 11, 1847 – October 18, 1931) was an American inventor and businessman. He developed many devices in fields such as electric power generation, mass communication, sound recording, and motion pictures. These inventions, which include the phonograph, the motion picture camera, and early versions of the electric light bulb, have had a widespread impact on the modern indust'},
 {'page_name': 'Thomas Edison State University',
  'pageid': '295279',
  'store_document_uri': 'wiki:295279',
  'url': 'https://en.wikipedia.org/wiki/Thomas_Edison_State_University',
  'summary': "Thomas Edison State University (TESU) is a public university in Trenton, New Jersey. The university is one of New Jersey's 11 senior public institutions of higher education. Thomas Edison State University offers degrees at the undergradua

In [67]:
tool.store_document(source='wiki:29778')

{'title': 'Thomas Alva Edison (February 11, 1847 – October 18, 1931) was an American inventor and businessman. He developed many devices in fields such as electric power generation, mass communication, sound recording, and motion pictures. These inventions, which include the phonograph, the motion picture',
 'source': 'wiki:29778',
 'chunks': 26}

In [70]:
tool.search_documents('light blub', 2)

{'chunk': 6,
 'description': 'Thomas Alva Edison (February 11, 1847 – October 18, 1931) was an American inventor and businessman. He developed many devices in fields such as electric power generation, mass communication, sound recording, and motion pictures. These inventions, whi',
 'id': '198',
 'source': 'wiki:29778',
 'text': 'In 1878, Edison began working on a system of electrical illumination, something he hoped could compete with gas and oil-based lighting. He began by tackling the problem of creating a long-lasting incandescent lamp, something that would be needed for indoor use. However, Thomas Edison did\nnot invent the light bulb. In 1840, British scientist Warren de la Rue developed an efficient light bulb using a coiled platinum filament but the high cost of platinum kept the bulb from becoming a commercial success. Many other inventors had also devised incandescent lamps,\nincluding Alessandro Volta\'s demonstration of a glowing wire in 1800 and inventions by Henry Woodwar